In [1]:
import pandas as pd
import cupy
import cupyx
import numpy as np
import swifter

In [2]:
val = pd.read_hdf('cleaned.h5', 'val')
train = pd.read_hdf('cleaned.h5', 'train')
candid = pd.read_hdf('cleaned.h5', 'candid')
candid = candid.reset_index(drop=True)

In [3]:
train = train.replace('\\n','', regex=True)
train = train.replace('\\r','', regex=True)
candid = candid.replace('\\n','', regex=True)
candid = candid.replace('\\r','', regex=True)
val = val.replace('\\n','', regex=True)
val = val.replace('\\r','', regex=True)

In [4]:
12

12

In [5]:
candid.abstract = candid.abstract.swifter.allow_dask_on_strings().apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))
candid.title = candid.title.swifter.allow_dask_on_strings().apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))

# train.description_text = train.description_text.apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))
# val.description_text = val.description_text.apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))

In [6]:
bm25_rank_train = np.load('train_recall3.0_0.75.npy') # or ber
bm25_rank_val = np.load('val_recall.npy')

In [7]:
bm25_rank_train.shape

(62975, 10000)

In [8]:
# check the recall rate in topk
def check_recall_rate(k = 10):
    idx2pid = candid['paper_id'].to_dict()
    pid2idx = {v: k for k, v in idx2pid.items()}
    
    cnt = 0
    tot = 0
    not_found = 0
    isin_top3 = 0
    map_score = 0
    for idx,row in enumerate(bm25_rank_train):
        row = row[:k]
        paper_id = train.iloc[idx].paper_id
        
        if paper_id not in pid2idx:
            not_found +=1
            continue
        correct_id = pid2idx[paper_id]
        
        if correct_id in row:
            cnt += 1
        if correct_id in row[:3]:
            isin_top3 += 1
            if correct_id == row[0]:
                map_score += 1.
            elif correct_id == row[1]:
                map_score += .5
            elif correct_id == row[2]:
                map_score += 1/3.
        tot += 1
    print('recall:', round(cnt / tot * 100,4) ,
          '\nnot found:', not_found,
          '\nin top3 rate:', round(isin_top3 / tot * 100,4),
          '\nmap:', round(map_score / tot,4))
    

In [9]:
"""
bm25_2.0_0.75
recall: 48.307 
not found: 897 
in top3 rate: 39.2796 
map: 0.339

bm25_1.2_0.75
recall: 49.085 
not found: 897 
in top3 rate: 40.2365 
map: 0.349

bm25_0.7_0.75
recall: 48.9143 
not found: 897 
in top3 rate: 40.338 
map: 0.352

bm25_1.2_0.87
recall: 49.3331 
not found: 897 
in top3 rate: 40.3895 
map: 0.3505
"""
check_recall_rate(k=10)

recall: 46.9828 
not found: 897 
in top3 rate: 37.9796 
map: 0.3255


In [10]:
"""
bm25_2.0_0.75
recall: 53.1622 
not found: 897 
in top3 rate: 39.2796 
map: 0.339

bm25_1.2_0.75
recall: 53.7485 
not found: 897 
in top3 rate: 40.2365 
map: 0.349
"""
check_recall_rate(k=20)

recall: 52.1022 
not found: 897 
in top3 rate: 37.9796 
map: 0.3255


In [11]:
"""
bm25_2.0_0.75
recall: 56.0295 
not found: 897 
in top3 rate: 39.2796 
map: 0.339

bm25_1.2_0.75
recall: 56.4934 
not found: 897 
in top3 rate: 40.2365 
map: 0.349

"""
check_recall_rate(k=30)

recall: 55.0662 
not found: 897 
in top3 rate: 37.9796 
map: 0.3255


In [12]:
"""
bm25_2.0_0.75
recall: 57.9142 
not found: 897 
in top3 rate: 39.2796 
map: 0.339

bm25_1.2_0.75
recall: 58.3798 
not found: 897 
in top3 rate: 40.2365 
map: 0.349

"""
check_recall_rate(k=40)

recall: 57.0138 
not found: 897 
in top3 rate: 37.9796 
map: 0.3255


In [13]:
"""
bm25_2.0_0.75
recall: 59.3399 
not found: 897 
in top3 rate: 39.2796 
map: 0.339

bm25_1.2_0.75
recall: 59.749 
not found: 897 
in top3 rate: 40.2365 
map: 0.349

"""
check_recall_rate(k=50)

recall: 58.4925 
not found: 897 
in top3 rate: 37.9796 
map: 0.3255


In [14]:
"""
bm25_2.0_0.75
recall: 63.8084 
not found: 897 
in top3 rate: 39.2796 
map: 0.339

bm25_1.2_0.75
recall: 64.1048 
not found: 897 
in top3 rate: 40.2365 
map: 0.349

"""
check_recall_rate(k=100)

recall: 63.0964 
not found: 897 
in top3 rate: 37.9796 
map: 0.3255


In [15]:
check_recall_rate(k=500)

recall: 72.4975 
not found: 897 
in top3 rate: 37.9796 
map: 0.3255


In [16]:
"""
bm25_2.0_0.75
recall: 76.0656 
not found: 897 
in top3 rate: 39.2796 
map: 0.339

bm25_1.2_0.75
recall: 76.0366 
not found: 897 
in top3 rate: 40.2365 
map: 0.349

bm25_0.7_0.75
"""
check_recall_rate(k=1000)

recall: 75.861 
not found: 897 
in top3 rate: 37.9796 
map: 0.3255


In [17]:
check_recall_rate(k=10000)

recall: 84.0072 
not found: 897 
in top3 rate: 37.9796 
map: 0.3255


In [7]:
# Generator Train

In [7]:
def gen_trainset(k=20):
    idx2pid = candid['paper_id'].to_dict()
    pid2idx = {v: k for k, v in idx2pid.items()}
    train_set = []
    train_val_set = []
    
    des = train.description_text.to_list()
    tit = candid.title.to_list()
    abst = candid.abstract.to_list()
    
    for idx,row in enumerate(bm25_rank_train):
        row = row[:k]
        paper_id = train.iloc[idx].paper_id
        
        if paper_id not in pid2idx:
            continue
            # 无答案
        
        correct_id = pid2idx[paper_id]
        if correct_id not in row:
            continue
            # 答案未被召回
            
        for pid in row:
            
            t = {
                'relavence': int(correct_id == pid),
                'qidx': idx,
                'pidx': pid,
                'q_text': des[idx],
                'd_text': tit[pid] + ' ' + abst[pid]
            }
            
            if (idx < 62000):
                if correct_id == pid:
                    for i in range(k - 1): # balance
                        train_set.append(t)
                else:
                    train_set.append(t)
            else:
                if correct_id == pid:
                    for i in range(k - 1): # balance
                        train_val_set.append(t)
                else:
                    train_val_set.append(t)
            
        if (idx % 1000 == 0):
            print(idx)
    
    pd.DataFrame(train_set).to_csv('bert_train.tsv', sep='\t', index=False)
    pd.DataFrame(train_val_set).to_csv('bert_train_val.tsv', sep='\t', index=False)
gen_trainset(k=20)

0
2000
3000
4000
6000
7000
9000
10000
12000
13000
14000
16000
17000
21000
24000
25000
27000
33000
35000
41000
42000
45000
48000
50000
51000
53000
54000
55000
56000
57000
59000
61000
62000


In [8]:
1

1

In [12]:
1

1

In [208]:
idx = 50113
print('recall_list', bm25_rank_train[idx,:10])
idx2pid = candid['paper_id'].to_dict()
pid2idx = {v: k for k, v in idx2pid.items()}
print('is_recalled:', pid2idx[train.iloc[idx].paper_id] in bm25_rank_train[idx,:10])
print('ans:', pid2idx[train.iloc[idx].paper_id])

recall_list [399651 445059  31071 409500  86217 245199 626399 577599 428215 261118]
is_recalled: True
ans: 626399


In [215]:
idx = 50123
print('recall_list', bm25_rank_train[idx,:10])
idx2pid = candid['paper_id'].to_dict()
pid2idx = {v: k for k, v in idx2pid.items()}
print('is_recalled:', pid2idx[train.iloc[idx].paper_id] in bm25_rank_train[idx,:10])
print('ans:', pid2idx[train.iloc[idx].paper_id])

recall_list [526064 452238 399427 424030 500370 623774 543678 415005 469279 437445]
is_recalled: True
ans: 437445


In [223]:
idx = 50138
print('recall_list', bm25_rank_train[idx,:10])
idx2pid = candid['paper_id'].to_dict()
pid2idx = {v: k for k, v in idx2pid.items()}
print('is_recalled:', pid2idx[train.iloc[idx].paper_id] in bm25_rank_train[idx,:10])
print('ans:', pid2idx[train.iloc[idx].paper_id])

recall_list [475310 509216  12398 366430 564468 376736 195028 269660 139719 538495]
is_recalled: True
ans: 12398


In [251]:
bm25_rank_val[8][:50]

array([ 22181,  21122,  10172, 262537,  84151, 478581,  48248, 157013,
        17607,  19681,  16816, 581586,  19688,  66812, 524472,  22182,
        19694, 636020, 542709,  15159,  59167,  10960,   8837, 338248,
         4026, 353713, 551293, 331105,  32562,  19691, 522166,   2880,
         1178,  22183, 461102,  28832, 531112,  33887,  16071,   8121,
        11543, 522513, 363646, 189090,  67595, 170588, 607793,  19693,
       280454,   8119], dtype=int64)

In [10]:
candid.iloc[59167].to_dict()

{'abstract': 'Different homozygous lines of similar genotype with respect to G6pd and 6Pgd were shown to have different enzyme activities for G6PD and 6PGD. Crosses between high and low lines suggested that there were modifying genes present on the autosomes, while others were probably located on the X chromosome. Allelic variation within each electrophoretic class of G6pd and 6Pgd might, however, also have contributed to this variation. An experiment on adaptation to sodium octanoate demonstrated that in adapted flies selection for lower enzyme activity had occurred, which provided further evidence for the existence of genetic differences in activity. Furthermore, a strong positive correlation between the activities of G6PD and 6PGD was found for each genotype. Since no correlation was found between MDH and the two enzymes G6PD and 6PGD, it could be concluded that this correlation was probably rather specific for G6PD and 6PGD. Interaction between genotypes with respect to activity wa

In [258]:
candid.iloc[8837].to_dict()

{'abstract': "The severity of enzyme deficiency often does not correlate well with the clinical severity of genetic diseases. Thus, some G6PD variants associated with severe enzyme deficiency, such as Union and Markham, cause no hemolytic problem, while some variants associated with less severe deficiency, such as Manchester, Alhambra, and Tripler, cause chronic hemolytic anemia. The kinetic characteristics of these variant enzymes have not explained the discrepancy. However, examination of the normal and variant enzymes under simulated physiologic conditions, with the effects of various intermediate metabolites and co-enzymes in red cells being taken into consideration, reveal that the G6PD's from hemolytic variant subjects are strongly inhibited by a physiologic concentration of NADPH because of their high Michaelis constant for NADP or low inhibition constant for NADPH, and they are more sensitive to inhibition by ATP. These variant enzymes cannot generate enough NADPH in red cells 